In [ ]:
#| hide

%load_ext autoreload
%autoreload 2

# Main

> Here, we will define the main functionalities of the package.

In [ ]:
#| default_exp main

In [ ]:
#| export
#| hide

import argparse
from thucy.configuration import *

def handle_config_command(args):
    if args.config_action == 'set':
        config.set_env_var(args.key, args.value)
    elif args.config_action == 'show':
        config.show_config()
    else:
        print("Use: thucy config [set|show]")

def main():
    parser = argparse.ArgumentParser(description='Thucy CLI')
    subparsers = parser.add_subparsers(dest='command', help='Commands')
    
    # Config command
    config_parser = subparsers.add_parser('config', help='Manage configuration')
    config_subparsers = config_parser.add_subparsers(dest='config_action')
    
    # thucy config set KEY VALUE
    set_parser = config_subparsers.add_parser('set', help='Set a config value')
    set_parser.add_argument('key', help='Config key (e.g., OPENAI_API_KEY)')
    set_parser.add_argument('value', help='Config value')
    
    # thucy config list
    config_subparsers.add_parser('show', help='List available keys')
    
    args = parser.parse_args()
    
    if args.command == 'config':
        handle_config_command(args)

In [ ]:
#| export

try: from nbdev.imports import IN_NOTEBOOK
except: IN_NOTEBOOK=False

In [ ]:
#| export
#| notest
if __name__ == '__main__' and not IN_NOTEBOOK:
    main()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()